In [56]:
from module.delivery_model import DeliveryModel
import numpy as np 

In [57]:
deliveryModel = DeliveryModel()

In [58]:
u=(13.840180,100.542326)
v=(13.803957,100.513704)
locations = np.array([(u,v),(u,v)])
# predictedDuration = deliveryModel.predict(u,v)[0]
# EucDist = np.apply_along_axis(lambda p :p[0] , axis=1, arr=locations)
# print(EucDist)
duration = deliveryModel.batch_predict(locations=locations,day_of_week=[])
print(duration)
# print(predictedDuration) 

[26.56447434 26.56447434]


/home/boatinw99/.local/lib/python3.8/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


In [5]:
np.full(5,"ONE")

array(['ONE', 'ONE', 'ONE', 'ONE', 'ONE'], dtype='<U3')

In [69]:
n = 4
X = np.empty(shape=[n, 0])

In [70]:
print(X)

[]


In [71]:
print(np.concatenate((X, b.T), axis=1))

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 4 and the array at index 1 has size 3

In [85]:
a = np.array([11,12,13])

print(a.T)
# X = np.apply_along_axis(lambda x :[x] , axis=1, arr=a)

[11 12 13]


AxisError: axis 1 is out of bounds for array of dimension 1

In [86]:
print(X)

[[11 12 13]]


In [2]:
# To use this module to predict the travel time from u -> v where u = (lat,long), v = (lat,long) : 
# 
# from module.delivery_model import DeliveryModel
# deliveryModel = DeliveryModel()
# u=(13.840180,100.542326) 
# v=(13.803957,100.513704)
# predictedDuration = deliveryModel.predict(u,v)[0]
# 
# predictedDuration will be the total time in minute 
# 

import pickle as pkl
import numpy as np
from math import * 
from module.distance_calculator import DistanceCalculator 
# from distance_calculator import DistanceCalculator 
from sklearn.ensemble import GradientBoostingRegressor

class DeliveryModel: 
    def __init__(self):
        self.model = pkl.load(open('gbdt_m_delivery.pkl', 'rb'))
        self.distanceCalculator = DistanceCalculator()
        self.dayMap = {"MON":0, "TUE":1, "WED":2, "THU":3, "FRI":4, "SAT":5, "SUN":6}

    def get_euc(self, coords_1, coords_2):
        R = 6371000
        conversion_const = 0.0174533
        c_1 = coords_1*conversion_const
        c_2 = coords_2*conversion_const
        delta_phi = abs(c_1[:,1]-c_2[:,1])
        theta = c_1[:,0]
        delta_theta = abs(c_1[:,0]-c_2[:,0])
        del_x = R*np.cos(theta)*delta_phi 
        del_y = R*delta_theta
        return np.sqrt(del_x**2 + del_y**2)
    
    def batch_predict(self, locations, day_of_week=[], approx=True):
        '''
        input:
        - locations = numpy array [(u_i,v_i) | i] where u_i = (lat,long) of the start location, 
        v_i = (lat_long) of the destination location 
        - (optional) day_of_week = numpy array [day_i | i] : choose one of these -> "MON", "TUE, "WED", "THU", "FRI", "SAT", "SUN"
        
        output:
        - The array prediction of the duration in minute (m) needed to travel from point u to point v 
        (please note that if you want to predict the duration needed from rider currently at point x and need to pick up food at point y and deliver at point z, you need to calculate the time it takes for x to go to y and the time it takes from a rider to from y to z)
        
        Note: 
        - When you set 'approx' to True, the model will approximate certain input feature for the sake of performance (speed) of prediction.
        - When you set 'approx' to False, the model will compute certain input feature more accurately but in expense of the speed of the prediciton.
        
        '''
        n = len(locations)
        idx = np.arange(0,n)
        if len(day_of_week) != n : 
            day_of_week = np.full(n,"MON")
        u = np.apply_along_axis(lambda loc_i : loc_i[0], axis=1, arr=locations)
        v = np.apply_along_axis(lambda loc_i : loc_i[1], axis=1, arr=locations)
        
        
        merchant_lat = np.apply_along_axis(lambda u_i : u_i[0], axis=1, arr=u)
        merchant_long = np.apply_along_axis(lambda u_i : u_i[1], axis=1, arr=u)
        
        customer_lat = np.apply_along_axis(lambda v_i : v_i[0], axis=1, arr=v)
        customer_long = np.apply_along_axis(lambda v_i : v_i[1], axis=1, arr=v)
    
        f = (lambda idx: self.get_euc(coords_1=u[idx], coords_2=v[idx]))
        
        EucDist = f(idx)
        if approx : 
            ShortestDist = EucDist.copy()*1.2
        else :
            ShortestDist = [self.distanceCalculator.shortestDistance(u[i],v[i]) for i in range(n)]

        u,inv = np.unique(day_of_week, return_inverse = True)
        day_inverse = np.array([self.dayMap[x] for x in u])[inv].reshape(day_of_week.shape)
        day_of_week_sin = np.apply_along_axis(lambda day : np.sin(day*(2.*np.pi/7)) , axis=0, arr=day_inverse)
        day_of_week_cos = np.apply_along_axis(lambda day : np.cos(day*(2.*np.pi/7)) , axis=0, arr=day_inverse)
        
        X = np.column_stack((merchant_lat, merchant_long, customer_lat, customer_long, EucDist, ShortestDist, day_of_week_sin, day_of_week_cos))
        return self.model.predict(X)
    

In [8]:
import time 
from module.delivery_model import DeliveryModel

In [9]:
deliveryModel = DeliveryModel()
u=(13.840180,100.542326)
v=(13.803957,100.513704)
X = 100*[(u,v)]
locations = np.array(X)


/home/boatinw99/.local/lib/python3.8/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator DummyRegressor from version 1.1.1 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/boatinw99/.local/lib/python3.8/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.1.1 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/boatinw99/.local/lib/python3.8/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator GradientBoostingRegressor from version 1.1.1 when using version 1.2.0. This might lead to breaking co

In [10]:
st = time.time()
duration = deliveryModel.batch_predict(locations=locations,day_of_week=np.array(10*["MON"]),approx=True)
ed = time.time() 
print(duration)
print(ed-st)

[26.56447434 26.56447434 26.56447434 26.56447434 26.56447434 26.56447434
 26.56447434 26.56447434 26.56447434 26.56447434 26.56447434 26.56447434
 26.56447434 26.56447434 26.56447434 26.56447434 26.56447434 26.56447434
 26.56447434 26.56447434 26.56447434 26.56447434 26.56447434 26.56447434
 26.56447434 26.56447434 26.56447434 26.56447434 26.56447434 26.56447434
 26.56447434 26.56447434 26.56447434 26.56447434 26.56447434 26.56447434
 26.56447434 26.56447434 26.56447434 26.56447434 26.56447434 26.56447434
 26.56447434 26.56447434 26.56447434 26.56447434 26.56447434 26.56447434
 26.56447434 26.56447434 26.56447434 26.56447434 26.56447434 26.56447434
 26.56447434 26.56447434 26.56447434 26.56447434 26.56447434 26.56447434
 26.56447434 26.56447434 26.56447434 26.56447434 26.56447434 26.56447434
 26.56447434 26.56447434 26.56447434 26.56447434 26.56447434 26.56447434
 26.56447434 26.56447434 26.56447434 26.56447434 26.56447434 26.56447434
 26.56447434 26.56447434 26.56447434 26.56447434 26

/home/boatinw99/.local/lib/python3.8/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
